In [2322]:
out_dir = '/Volumes/Expansion/datasets/Manitoba_PET/PET_R_Maps/autonocog_memory/similarity_to_lesion_memory_net'

# 01 - Get Datasets


**Data will be Imported from a CSV which is expected to be in this format**
- sub column contents MUST match the names of the neuroimaging files above. 
    - ID column 
```
+-----+----------------------------+--------------+--------------+--------------+
| sub | Nifti_File_Path            | Indep. Var.  | Covariate_N  | Dataset      |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 1            |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 1            |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 2            |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3            |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

In [ ]:
input_csv_path = 'path/to/your/csv_file.csv'
sheet = None # Set to None if CSV

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
drop_list = ['variable_of_interest']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
display(data_df)

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [ ]:
column = ''
condition = ''
value = ''

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
data_df

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [ ]:
# Remove anything you don't want to standardize
cols_not_to_standardize = ['DX', 'SUBID'] # ['Z_Scored_Percent_Cognitive_Improvement_By_Origin_Group', 'Z_Scored_Subiculum_T_By_Origin_Group_'] #['Age']
group_col = '' #Set to none if there are no specific groups

In [ ]:
data_df = cal_palm.standardize_columns(cols_not_to_standardize, group_col)
data_df

# 02 - Import the Data into DataFrames

In [ ]:
dataset_col = 'Ds'
nifti_path_col = 'path'
indep_var_col = 'variable_of_interest'
covariate_cols = ['Age', 'sex']

In [ ]:
from calvin_utils.file_utils.import_functions import DatasetNiftiImporter
data_importer = DatasetNiftiImporter(df=data_df, dataset_col=dataset_col, nifti_col=nifti_path_col, indep_var_col=indep_var_col, covariate_cols=covariate_cols)
data_dict = data_importer.data_dict

# 03 - Control for Covariates

Regress out a Covariate
- This will control for a covariate by regressing it out of BOTH the independent variable and the niftis. 
- This will leave the residuals as the resulting data
- Make sure to standardize the niftis (if needed) and definitiely the indep var and covariates before running this. 

In [ ]:
from calvin_utils.statistical_utils.regression_utils import RegressOutCovariates
for dataset in data_dict.keys():
    data_dict[dataset]['niftis'] = RegressOutCovariates.regress_out_covariates_using_endog_exog(data_dict[dataset]['niftis'], data_dict[dataset]['covariates'])
    data_dict[dataset]['indep_var'] = RegressOutCovariates.regress_out_covariates_using_endog_exog(data_dict[dataset]['indep_var'], data_dict[dataset]['covariates'])

# 04 - Begin Analysis